In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.cuda import device

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
input_size = 32*3
hidden_size = 128
output_size = 10


In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
def preprocess_images(image):
    return image.view(image.size(0), 32, -1)

In [7]:
class RnnClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RnnClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out
    
    

        

In [8]:
model = RnnClassifier(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RnnClassifier(
  (rnn): RNN(96, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

In [13]:
num_epochs = 20
model.train()
for epoch in range(num_epochs):
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        images = preprocess_images(images)
        
        output = model(images)
        loss = criterion(output, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.7262
Epoch [2/20], Loss: 1.6800
Epoch [3/20], Loss: 1.3467
Epoch [4/20], Loss: 1.2007
Epoch [5/20], Loss: 1.5891
Epoch [6/20], Loss: 1.3017
Epoch [7/20], Loss: 0.9723
Epoch [8/20], Loss: 1.3415
Epoch [9/20], Loss: 1.5153
Epoch [10/20], Loss: 1.2121
Epoch [11/20], Loss: 1.4528
Epoch [12/20], Loss: 0.8860
Epoch [13/20], Loss: 1.2968
Epoch [14/20], Loss: 1.1757
Epoch [15/20], Loss: 1.0293
Epoch [16/20], Loss: 1.0023
Epoch [17/20], Loss: 1.7847
Epoch [18/20], Loss: 1.0869
Epoch [19/20], Loss: 1.0495
Epoch [20/20], Loss: 1.1372


In [14]:
def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            images = preprocess_images(images)  # Chuyển ảnh thành chuỗi
            
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Độ chính xác trên tập test: {accuracy:.2f}%')

evaluate()

Độ chính xác trên tập test: 46.15%
